In [1]:
import torch
import torch.nn as nn
import time
import random
import os

### Ultis

In [2]:
import json
import os
import pandas as pd

special_char = {
    "x": "xs",
    "s": "sx",
    "d": "drg",
    "r": "rdg",
    "g": "grd",
    "ch": "tr",
    "tr": "ch",
    "n": "l",
    "l": "n",
}

vietnamese_characters = [
    "á",
    "à",
    "ả",
    "ã",
    "ạ",
    "ă",
    "ắ",
    "ằ",
    "ẳ",
    "ẵ",
    "ặ",
    "â",
    "ấ",
    "ầ",
    "ẩ",
    "ẫ",
    "ậ",
    "é",
    "è",
    "ẻ",
    "ẽ",
    "ẹ",
    "ê",
    "ế",
    "ề",
    "ể",
    "ễ",
    "ệ",
    "í",
    "ì",
    "ỉ",
    "ĩ",
    "ị",
    "ó",
    "ò",
    "ỏ",
    "õ",
    "ọ",
    "ô",
    "ố",
    "ồ",
    "ổ",
    "ỗ",
    "ộ",
    "ơ",
    "ớ",
    "ờ",
    "ở",
    "ỡ",
    "ợ",
    "ú",
    "ù",
    "ủ",
    "ũ",
    "ụ",
    "ư",
    "ứ",
    "ừ",
    "ử",
    "ữ",
    "ự",
    "ý",
    "ỳ",
    "ỷ",
    "ỹ",
    "ỵ",
    "đ",
]
# sac, huyen, hoi, nga, nang, mu, rau, mu nguoc :>
special_token_sign_vietnamese = {
    "sac": "<´>",
    "huyen": "<`>",
    "hoi": "<?>",
    "nga": "<~>",
    "nang": "<.>",
    "mu": "<^>",
    "rau": "<,>",
    "mu_nguoc": "<^^>",
}
convert_char_vietnamese = {
    "á": ("a", special_token_sign_vietnamese["sac"]),
    "à": ("a", special_token_sign_vietnamese["huyen"]),
    "ả": ("a", special_token_sign_vietnamese["hoi"]),
    "ã": ("a", special_token_sign_vietnamese["nga"]),
    "ạ": ("a", special_token_sign_vietnamese["nang"]),
    "ă": ("a", special_token_sign_vietnamese["mu_nguoc"]),
    "ắ": (
        "a",
        special_token_sign_vietnamese["mu_nguoc"],
        special_token_sign_vietnamese["sac"],
    ),
    "ằ": (
        "a",
        special_token_sign_vietnamese["mu_nguoc"],
        special_token_sign_vietnamese["huyen"],
    ),
    "ẳ": (
        "a",
        special_token_sign_vietnamese["mu_nguoc"],
        special_token_sign_vietnamese["hoi"],
    ),
    "ẵ": (
        "a",
        special_token_sign_vietnamese["mu_nguoc"],
        special_token_sign_vietnamese["nga"],
    ),
    "ặ": (
        "a",
        special_token_sign_vietnamese["mu_nguoc"],
        special_token_sign_vietnamese["nang"],
    ),
    "â": ("a", special_token_sign_vietnamese["mu"]),
    "ấ": (
        "a",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["sac"],
    ),
    "ầ": (
        "a",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["huyen"],
    ),
    "ẩ": (
        "a",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["hoi"],
    ),
    "ẫ": (
        "a",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["nga"],
    ),
    "ậ": (
        "a",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["nang"],
    ),
    "é": ("e", special_token_sign_vietnamese["sac"]),
    "è": ("e", special_token_sign_vietnamese["huyen"]),
    "ẻ": ("e", special_token_sign_vietnamese["hoi"]),
    "ẽ": ("e", special_token_sign_vietnamese["nga"]),
    "ẹ": ("e", special_token_sign_vietnamese["nang"]),
    "ê": ("e", special_token_sign_vietnamese["mu"]),
    "ế": (
        "e",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["sac"],
    ),
    "ề": (
        "e",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["huyen"],
    ),
    "ể": (
        "e",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["hoi"],
    ),
    "ễ": (
        "e",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["nga"],
    ),
    "ệ": (
        "e",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["nang"],
    ),
    "í": ("i", special_token_sign_vietnamese["sac"]),
    "ì": ("i", special_token_sign_vietnamese["huyen"]),
    "ỉ": ("i", special_token_sign_vietnamese["hoi"]),
    "ĩ": ("i", special_token_sign_vietnamese["nga"]),
    "ị": ("i", special_token_sign_vietnamese["nang"]),
    "ó": ("o", special_token_sign_vietnamese["sac"]),
    "ò": ("o", special_token_sign_vietnamese["huyen"]),
    "ỏ": ("o", special_token_sign_vietnamese["hoi"]),
    "õ": ("o", special_token_sign_vietnamese["nga"]),
    "ọ": ("o", special_token_sign_vietnamese["nang"]),
    "ô": ("o", special_token_sign_vietnamese["mu"]),
    "ố": (
        "o",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["sac"],
    ),
    "ồ": (
        "o",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["huyen"],
    ),
    "ổ": (
        "o",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["hoi"],
    ),
    "ỗ": (
        "o",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["nga"],
    ),
    "ộ": (
        "o",
        special_token_sign_vietnamese["mu"],
        special_token_sign_vietnamese["nang"],
    ),
    "ơ": ("o", special_token_sign_vietnamese["rau"]),
    "ớ": (
        "o",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["sac"],
    ),
    "ờ": (
        "o",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["huyen"],
    ),
    "ở": (
        "o",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["hoi"],
    ),
    "ỡ": (
        "o",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["nga"],
    ),
    "ợ": (
        "o",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["nang"],
    ),
    "ú": ("u", special_token_sign_vietnamese["sac"]),
    "ù": ("u", special_token_sign_vietnamese["huyen"]),
    "ủ": ("u", special_token_sign_vietnamese["hoi"]),
    "ũ": ("u", special_token_sign_vietnamese["nga"]),
    "ụ": ("u", special_token_sign_vietnamese["nang"]),
    "ư": ("u", special_token_sign_vietnamese["rau"]),
    "ứ": (
        "u",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["sac"],
    ),
    "ừ": (
        "u",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["huyen"],
    ),
    "ử": (
        "u",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["hoi"],
    ),
    "ữ": (
        "u",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["nga"],
    ),
    "ự": (
        "u",
        special_token_sign_vietnamese["rau"],
        special_token_sign_vietnamese["nang"],
    ),
    "ý": ("y", special_token_sign_vietnamese["sac"]),
    "ỳ": ("y", special_token_sign_vietnamese["huyen"]),
    "ỷ": ("y", special_token_sign_vietnamese["hoi"]),
    "ỹ": ("y", special_token_sign_vietnamese["nga"]),
    "ỵ": ("y", special_token_sign_vietnamese["nang"]),
    "đ": ("d", "d"),
}


def write_dict_data_to_file(file_path, data, indent=2):
    with open(file_path, "w", encoding="utf-8") as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=indent)


def write_list_data_to_file(file_path, list_data):
    # os.makedirs(file_path, exist_ok=True)
    with open(file=file_path, mode="w", encoding="utf-8") as file:
        file.write("".join([x + "\n" for x in list_data]))


In [3]:

import pandas as pd
import string


def load_data_and_vocab(
    path_file_data=None,
    data_size=1000,
):
    file = open(path_file_data, "r")
    sentences = file.readlines()
    sentences = [
        replace_punctuation(sentence.replace("\n", "").lower())
        for sentence in sentences
    ]
    return sentences[:data_size]


def replace_punctuation(text, replacement=""):
    translator = str.maketrans({key: replacement for key in string.punctuation})
    return text.translate(translator)


In [4]:
import collections
import re
import unicodedata
import six


class CreateVocab(object):
    def __init__(self, special_token):
        self.special_token = special_token

    def create_word_vocab(self, data_text, vocab_size=None):
        print("Creating vocab:")
        word_vocab = collections.OrderedDict()
        list_word_token = collections.OrderedDict()
        for sentence in data_text:
            list_token = convert_sentence_to_token(sentence)
            for token in list_token:
                if contains_number(token) or contains_special_character(token):
                    continue
                if token in list_word_token:
                    list_word_token[token] = list_word_token[token] + 1
                else:
                    list_word_token[token] = 1
        list_word_token = collections.OrderedDict(
            sorted(list_word_token.items(), key=lambda x: x[1], reverse=True)
        )
        index = 0
        for token in self.special_token:
            word_vocab[token] = index
            index += 1
        for token in list_word_token:
            word_vocab[token] = index
            if vocab_size != None:
                if index > vocab_size - len(self.special_token):
                    break
            index += 1
        return word_vocab

    def create_character_vocab(self, file_text, vocab_size=None):
        character_vocab = collections.OrderedDict()
        list_char_token = collections.OrderedDict()
        for senetence in file_text:
            for char in senetence:
                if contains_special_character(token=char):
                    continue
                if char in list_char_token:
                    list_char_token[char] = list_char_token[char] + 1
                else:
                    list_char_token[char] = 1
        list_char_token = collections.OrderedDict(
            sorted(list_char_token.items(), key=lambda x: x[1], reverse=True)
        )
        index = 0
        for token in self.special_token:
            character_vocab[token] = index
            index += 1
        for token in list_char_token:
            if token in vietnamese_characters:
                continue
            character_vocab[token] = index
            if vocab_size != None:
                if index > vocab_size - len(self.special_token):
                    break
            index += 1
        for token in special_token_sign_vietnamese.values():
            character_vocab[token] = index
            index += 1
        return character_vocab


def contains_special_character(token):
    for char in token:
        if _is_whitespace(char=char):
            return True
        # if _is_punctuation(char=char):
        #     return True
    return False


def contains_number(text):
    pattern = re.compile(r"[0-9]")
    return bool(pattern.search(text))


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def convert_sentence_to_token(text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
        # if self.do_lower_case:
        token = token.lower()
        # token = self._run_strip_accents(token)
        split_tokens.extend(run_split_on_punc(token))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens


def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))


def run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1

    return ["".join(x) for x in output]


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (
        (cp >= 33 and cp <= 47)
        or (cp >= 58 and cp <= 64)
        or (cp >= 91 and cp <= 96)
        or (cp >= 123 and cp <= 126)
    ):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat in ("Cc", "Cf"):
        return True
    return False


In [5]:
from torch.nn.utils.rnn import pad_sequence
import torch


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, char_input_ids, word_input_ids, mask_token_positions, target):
        self.char_input_ids = char_input_ids
        self.word_input_ids = word_input_ids
        self.mask_token_positions = mask_token_positions
        self.target = target

    def __getitem__(self, idx):
        target = self.target[idx]
        word_input_ids = self.word_input_ids[idx]
        mask_token_positions = self.mask_token_positions[idx]
        char_input_ids = self.char_input_ids[idx]

        return {
            "word_input_ids": word_input_ids,
            "mask_token_positions": mask_token_positions,
            "char_input_ids": char_input_ids,
        }, target

    def __len__(self):
        return len(self.target)

    def collate_fn(self, batch):
        word_input_ids = [item[0]["word_input_ids"] for item in batch]
        mask_token_positions = [item[0]["mask_token_positions"] for item in batch]
        char_input_ids = [item[0]["char_input_ids"] for item in batch]
        targets = [item[1] for item in batch]

        # Pad sequences
        word_input_ids_padded = pad_sequence(
            word_input_ids, batch_first=True, padding_value=0
        )
        mask_token_positions_padded = pad_sequence(
            mask_token_positions, batch_first=True, padding_value=0
        )
        char_input_ids_padded = pad_sequence(
            char_input_ids, batch_first=True, padding_value=0
        )

        return {
            "word_input_ids": word_input_ids_padded,
            "mask_token_positions": mask_token_positions_padded,
            "char_input_ids": char_input_ids_padded,
        }, targets


In [6]:
import os
import torch
import re
import collections
import unicodedata
import six


class Tokenizer(object):
    """Runs end-to-end tokenziation."""

    def __init__(
        self, vocab_file, unk_token="[UNK]", do_lower_case=True, char_level=False
    ):
        self.vocab = load_vocab(vocab_file)
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.do_lower_case = do_lower_case
        self.char_level = char_level
        self.unk_token = unk_token

    def tokenize(self, text):
        if self.char_level:
            return self.__tokenize_char(text=text)
        else:
            return self.__tokenize_word(text=text)

    def __tokenize_word(self, text):
        text = convert_to_unicode(text)
        text = self._clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case:
                token = token.lower()
            split_tokens.extend(self._run_split_on_punc(token))
        return [
            token if token in self.vocab else self.unk_token
            for token in whitespace_tokenize(" ".join(split_tokens))
        ]

    def __tokenize_char(self, text):
        text = convert_to_unicode(text)
        text = self._clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case:
                token = token.lower()
            split_tokens.extend(self._run_split_on_punc(token))
        output = []
        for token in whitespace_tokenize(" ".join(split_tokens)):
            char_output = []
            for char in token:
                if char in vietnamese_characters:
                    for spec_token in convert_char_vietnamese[char]:
                        char_output.append(spec_token)
                    continue
                char_output.append(char if char in self.vocab else self.unk_token)
            output.append(char_output)
        return output

    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xFFFD or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]

    def convert_tokens_to_ids(self, tokens):
        return convert_by_vocab(self.vocab, tokens)

    def convert_ids_to_tokens(self, ids):
        return convert_by_vocab(self.inv_vocab, ids)


def convert_by_vocab(vocab, items):
    """Converts a sequence of [tokens|ids] using the vocab."""
    output = []
    for item in items:
        output.append(vocab[item])
    return output


def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with open(file=vocab_file, mode="r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab


def contains_special_character(token):
    for char in token:
        if _is_whitespace(char=char):
            return True
        if _is_punctuation(char=char):
            return True
    return False


def contains_number(text):
    pattern = re.compile(r"[0-9]")
    return bool(pattern.search(text))


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def convert_sentence_to_token(text):
    """Tokenizes a piece of text."""
    text = convert_to_unicode(text)
    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
        # if self.do_lower_case:
        token = token.lower()
        # token = self._run_strip_accents(token)
        split_tokens.extend(run_split_on_punc(token))

    output_tokens = whitespace_tokenize(" ".join(split_tokens))
    return output_tokens


def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        # elif isinstance(text, unicode):
        #   return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")


def run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1

    return ["".join(x) for x in output]


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (
        (cp >= 33 and cp <= 47)
        or (cp >= 58 and cp <= 64)
        or (cp >= 91 and cp <= 96)
        or (cp >= 123 and cp <= 126)
    ):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat in ("Cc", "Cf"):
        return True
    return False


In [7]:
import collections
import unicodedata
from transformers import AutoTokenizer
import torch


class DataBuilderForElectra:
    def __init__(
        self,
        rng,
        seq_max_length,
        tokenizer,
    ):
        self.rng = rng
        self.tokenizer: AutoTokenizer = tokenizer
        self.seq_max_length = seq_max_length

    def build_data(self, inputs):
        input_original = []
        input_masked = []
        for sentence in inputs:
            tokens = split_token(sentence)
            tokens.insert(0, self.tokenizer.cls_token)
            if len(tokens) > self.seq_max_length:
                tokens = tokens[: self.seq_max_length]
            tokens_masked = create_masked_lm_predictions(
                tokens=tokens,
                masked_lm_prob=0.20,
                max_predictions_per_seq=3,
                rng=self.rng,
            )
            tokens = self.tokenizer.convert_tokens_to_ids(tokens)
            tokens_masked = self.tokenizer.convert_tokens_to_ids(tokens_masked)
            input_original.append(
                padding(
                    tokens, self.seq_max_length, padding_id=self.tokenizer.pad_token_id
                )
            )
            input_masked.append(
                padding(
                    tokens_masked,
                    self.seq_max_length,
                    padding_id=self.tokenizer.pad_token_id,
                )
            )
        return torch.Tensor(input_original).long(), torch.Tensor(input_masked).long()


def split_token(text):
    text = _clean_text(text)
    orig_tokens = whitespace_tokenize(text)
    split_tokens = []
    for token in orig_tokens:
        token = token.lower()
        split_tokens.extend(run_split_on_punc(token))
    return [token for token in whitespace_tokenize(" ".join(split_tokens))]


def padding(input_ids, max_length, padding_id):
    len_sentence = len(input_ids)
    while len(input_ids) > max_length:
        del input_ids[-1]
    for i in range(max_length - len_sentence):
        input_ids.append(padding_id)
    return input_ids


MaskedLmInstance = collections.namedtuple("MaskedLmInstance", ["index", "label"])


def create_masked_lm_predictions(
    tokens,
    masked_lm_prob,
    max_predictions_per_seq,
    rng,
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
):
    """Creates the predictions for the masked LM objective."""

    cand_indexes = []
    for i, token in enumerate(tokens):
        if token == cls_token or token == sep_token:
            continue
        cand_indexes.append([i])

    rng.shuffle(cand_indexes)

    output_tokens = list(tokens)

    num_to_predict = min(
        max_predictions_per_seq, max(1, int(round(len(tokens) * masked_lm_prob)))
    )

    masked_lms = []
    covered_indexes = set()
    for index_set in cand_indexes:
        if len(masked_lms) >= num_to_predict:
            break
        # If adding a whole-word mask would exceed the maximum number of
        # predictions, then just skip this candidate.
        if len(masked_lms) + len(index_set) > num_to_predict:
            continue
        is_any_index_covered = False
        for index in index_set:
            if index in covered_indexes:
                is_any_index_covered = True
                break
        if is_any_index_covered:
            continue
        for index in index_set:
            covered_indexes.add(index)
            masked_token = None
            masked_token = mask_token
            output_tokens[index] = masked_token
            masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))
    assert len(masked_lms) <= num_to_predict
    masked_lms = sorted(masked_lms, key=lambda x: x.index)

    masked_lm_positions = []
    masked_lm_labels = []
    for p in masked_lms:
        masked_lm_positions.append(p.index)
        masked_lm_labels.append(p.label)

    return output_tokens


def _clean_text(text):
    """Performs invalid character removal and whitespace cleanup on text."""
    output = []
    for char in text:
        cp = ord(char)
        if cp == 0 or cp == 0xFFFD or _is_control(char):
            continue
        if _is_whitespace(char):
            output.append(" ")
        else:
            output.append(char)
    return "".join(output)


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat in ("Cc", "Cf"):
        return True
    return False


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens


def run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1

    return ["".join(x) for x in output]


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (
        (cp >= 33 and cp <= 47)
        or (cp >= 58 and cp <= 64)
        or (cp >= 91 and cp <= 96)
        or (cp >= 123 and cp <= 126)
    ):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False


In [8]:
import collections
import random
import torch
# from utils import special_char


class DataBuilder:
    def __init__(
        self,
        rng,
        padding_id,
        mask_token,
        seq_max_length,
        word_max_length,
        custom_tokenizer: Tokenizer,
        char_tokenizer: Tokenizer,
    ):
        self.rng = rng
        self.custom_tokenizer = custom_tokenizer
        self.char_tokenizer = char_tokenizer
        self.word_max_length = word_max_length
        self.seq_max_length = seq_max_length
        self.padding_id = padding_id
        self.mask_token = mask_token

    def build_data(self, inputs):
        word_input_ids = []
        mask_token_positions = []
        target = []
        char_input_ids = []
        for seq in inputs:
            seq_char_ids = []
            seq_char = seq
            for word in seq_char.split(" "):
                for start_char in special_char.keys():
                    if word.startswith(start_char):
                        seq_char = seq_char.replace(
                            word, special_char[start_char] + word
                        )
            tokens = self.char_tokenizer.tokenize(seq_char)
            for word in tokens:
                word_char_ids = self.char_tokenizer.convert_tokens_to_ids(word)
                word_char_ids = padding(
                    word_char_ids, self.word_max_length, self.padding_id
                )
                seq_char_ids.append(word_char_ids)
            char_pad = padding([], self.word_max_length, self.padding_id)
            seq_char_ids = padding(seq_char_ids, self.seq_max_length, char_pad)
            tokens = self.custom_tokenizer.tokenize(seq)
            if len(tokens) > self.seq_max_length:
                for i in range(len(tokens) - self.seq_max_length):
                    list_tokens = tokens[i : i + self.seq_max_length]
                    token_masked, masked_positions, masked_label = (
                        create_masked_lm_predictions(
                            rng=self.rng,
                            tokens=list_tokens,
                            vocab_words=self.custom_tokenizer.inv_vocab,
                            mask_token=self.mask_token,
                            masked_lm_prob=0.15,
                            max_predictions_per_seq=5,
                        )
                    )
                    word_input_ids = word_input_ids + [
                        self.custom_tokenizer.convert_tokens_to_ids(_)
                        for _ in token_masked
                    ]
                    for _ in masked_positions:
                        __ = torch.zeros(self.seq_max_length, dtype=bool)
                        __[_] = True
                        mask_token_positions.append(__)
                    target = target + [
                        self.custom_tokenizer.convert_tokens_to_ids(_)
                        for _ in masked_label
                    ]
                    char_input_ids = char_input_ids + [
                        seq_char_ids for _ in range(len(token_masked))
                    ]
            else:
                token_masked, masked_positions, masked_label = (
                    create_masked_lm_predictions(
                        rng=self.rng,
                        tokens=tokens,
                        vocab_words=self.custom_tokenizer.inv_vocab,
                        mask_token=self.mask_token,
                        masked_lm_prob=0.15,
                        max_predictions_per_seq=5,
                    )
                )
                word_input_ids = word_input_ids + [
                    padding(
                        self.custom_tokenizer.convert_tokens_to_ids(_),
                        max_length=self.seq_max_length,
                        padding_id=1,
                    )
                    for _ in token_masked
                ]
                for _ in masked_positions:
                    __ = torch.zeros(self.seq_max_length, dtype=bool)
                    __[_] = True
                    mask_token_positions.append(__)
                target = target + [
                    self.custom_tokenizer.convert_tokens_to_ids(_) for _ in masked_label
                ]
                char_input_ids = char_input_ids + [
                    seq_char_ids for _ in range(len(token_masked))
                ]

        return (
            torch.tensor(char_input_ids),
            torch.tensor(word_input_ids),
            torch.stack(mask_token_positions),
            target,
        )


def padding(input_ids, max_length, padding_id):
    len_sentence = len(input_ids)
    while len(input_ids) > max_length:
        del input_ids[-1]
    for i in range(max_length - len_sentence):
        input_ids.append(padding_id)
    return input_ids


MaskedLmInstance = collections.namedtuple("MaskedLmInstance", ["index", "label"])


def create_masked_char_input(
    tokens: list, vocab_chars, mask_token, rng: random.Random, masked_lm_prob=0.25
):
    size_alter = int(len(tokens) * (1 + masked_lm_prob))
    number_range = list(range(size_alter))
    output = tokens
    cand_indexes = rng.sample(number_range, size_alter - len(output))
    cand_indexes.sort()
    for index in cand_indexes:
        random_char_token = vocab_chars[rng.randint(0, len(vocab_chars) - 1)]
        output.insert(index, random_char_token)
    output.append(mask_token)
    return output


def create_masked_lm_predictions(
    tokens: list,
    masked_lm_prob,
    max_predictions_per_seq,
    vocab_words: dict,
    rng,
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
):
    """Creates the predictions for the masked LM objective."""

    cand_indexes = []
    for i, token in enumerate(tokens):
        if token == cls_token or token == sep_token:
            continue
        cand_indexes.append(i)

    rng.shuffle(cand_indexes)

    output_tokens = []

    num_to_predict = min(
        max_predictions_per_seq, max(1, int(round(len(tokens) * masked_lm_prob)))
    )
    list_masked_lms = []
    list_masked_lm_positions = []
    list_masked_lm_labels = []
    covered_indexes = set()
    while len(covered_indexes) < len(cand_indexes):
        masked_lms = []
        tokens_masked = list(tokens)
        for index in cand_indexes:
            if index in covered_indexes:
                continue
            covered_indexes.add(index)

            masked_token = None
            # 80% of the time, replace with [MASK]
            if rng.random() < 0.8:
                masked_token = mask_token
            else:
                # 10% of the time, keep original
                if rng.random() < 0.5:
                    masked_token = tokens[index]
                # 10% of the time, replace with random word
                else:
                    masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]

            tokens_masked[index] = masked_token
            masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))
            if len(masked_lms) >= num_to_predict:
                masked_lms = sorted(masked_lms, key=lambda x: x.index)
                masked_lm_positions = []
                masked_lm_labels = []
                for p in masked_lms:
                    masked_lm_positions.append(p.index)
                    masked_lm_labels.append(p.label)
                break
        output_tokens.append(tokens_masked)
        list_masked_lms.append(masked_lms)
        list_masked_lm_positions.append(masked_lm_positions)
        list_masked_lm_labels.append(masked_lm_labels)

    return (output_tokens, list_masked_lm_positions, list_masked_lm_labels)

In [9]:
# from utils import write_list_data_to_file
import pandas as pd
import string


def load_data_and_vocab(
    path_file_data=None,
    data_size=1000,
):
    file = open(path_file_data, "r")
    sentences = file.readlines()
    sentences = [
        replace_punctuation(sentence.replace("\n", "").lower())
        for sentence in sentences
    ]
    return sentences[:data_size]


def replace_punctuation(text, replacement=""):
    translator = str.maketrans({key: replacement for key in string.punctuation})
    return text.translate(translator)


### Transformer

In [10]:
import torch
import torch.nn as nn
import numpy as np


def positional_encoding(length, depth):
    depth = depth // 2

    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :] / depth

    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates

    pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)

    return torch.tensor(pos_encoding, dtype=torch.float32)


class PositionalEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, positional=True):
        super(PositionalEmbedding, self).__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.positional = positional
        if positional:
            self.pos_encoding = positional_encoding(length=2048, depth=d_model)

    def forward(self, x):
        length = x.size(1)
        # x.size() return torch.Size([3200, 10])
        # if torch.max(x) >= self.embedding.num_embeddings:
        #     raise ValueError("Input indices are out of range for the embedding layer.")
        x = self.embedding(x)
        x = x * torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32)).to(x.device)
        if self.positional:
            x = x + self.pos_encoding[:length, :].unsqueeze(0).to(x.device)
        return x


In [11]:
from torch import nn


class SelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(SelfAttention, self).__init__()
        # self.mha = torch.nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads,dropout=dropout)
        self.mha = nn.MultiheadAttention(
            embed_dim=embed_dim, num_heads=num_heads, batch_first=True
        )
        self.layernorm = nn.LayerNorm(normalized_shape=embed_dim)

    # @torch.jit.unused(
    def forward(self, *inputs):
        # x (batch_size x seq_length x embed_dim), mask (batch_size . numhead x seq_length x seq_length)
        x, key_padding_mask = inputs
        (attn_output, attn_output_weight) = self.mha(
            query=x, value=x, key=x, key_padding_mask=key_padding_mask
        )
        x = x + attn_output
        x = self.layernorm(x)
        return x


In [12]:
import torch.nn as nn


class FeedForward(nn.Module):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super(FeedForward, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(),
            nn.Linear(dff, d_model),
            nn.Dropout(dropout_rate)
        )
        self.add = nn.Identity()
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        residual = x
        x = self.seq(x)
        x = self.add(residual + x)
        x = self.layer_norm(x)
        return x


In [13]:
import torch.nn as nn
import torch


class EncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout, dff):
        super(EncoderLayer, self).__init__()
        self.self_attention = SelfAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.fn = FeedForward(d_model=embed_dim, dff=dff)

    def forward(self, *input):
        x = self.self_attention(*input)
        x = self.fn(x)
        return x


### Model

In [14]:
import torch
from torch import nn

class EncoderBERTpretrain(nn.Module):
    def __init__(self, embedding_layer, encoder_layers, device) -> None:
        super().__init__()
        self.embedding_layer = embedding_layer.to(device)
        self.encoder_layers: nn.ModuleList = encoder_layers.to(device)

    def forward(self, input_phobert_ids):
        input_ids = input_phobert_ids
        attn_mask = input_ids != 1
        attn_mask = attn_mask[:, None, None, :]
        output = self.embedding_layer(input_ids)
        for layer in self.encoder_layers:
            output = layer(output, attention_mask=attn_mask)[0]
        return output


In [15]:
import torch
from torch import nn


class EncoderCharacter(nn.Module):
    def __init__(
        self,
        num_layers,
        num_heads,
        embed_dim,
        dff,
        vocab_size,
        device,
        dropout_rate,
    ) -> None:
        super().__init__()
        self.embedding_layer = PositionalEmbedding(
            vocab_size=vocab_size, d_model=embed_dim
        ).to(device=device)
        self.encoder_layers = nn.ModuleList(
            [
                EncoderLayer(
                    embed_dim=embed_dim,
                    num_heads=num_heads,
                    dropout=dropout_rate,
                    dff=dff,
                ).to(device)
                for i in range(num_layers)
            ]
        )
        self.device = device

    def forward(self, *inputs):
        # input_ids (batch_size x sentence_len x word_len)
        input_ids, word_padding = inputs
        batch_size, sentence_len, word_len = input_ids.size()
        # input_ids ((batch_size * sentence_len) x word_len)
        input_ids = input_ids.view(batch_size * sentence_len, word_len)
        word_padding = word_padding.view(batch_size * sentence_len)
        input_embed = input_ids[word_padding]
        output = self.embedding_layer(input_embed)
        key_mask_padding = input_embed == 1
        for layer in self.encoder_layers:
            output = layer(output, key_mask_padding)
        output_padding = torch.zeros(
            [batch_size * sentence_len, word_len, output.size(-1)]
        ).to(device=self.device)
        output_padding[word_padding] = output
        output_padding.reshape(
            batch_size, sentence_len, word_len, output_padding.size(-1)
        )
        output_padding = output_padding.mean(dim=1)
        return output_padding


In [16]:
import torch
import torch.nn as nn

# from custom_layer.EncoderCharacter import EncoderCharacter
# from custom_layer.EncoderBERTpretrain import EncoderBERTpretrain
# from transformer.EncoderLayer import EncoderLayer
# from transformer.PositionalEmbedding import PositionalEmbedding

# input: word_feature: (batch_size x sen_len x 768), chars_not_embedded: (batch_size x sen_len x word_len) -
# output: (batch_size x sen_len)


class ModelSC(nn.Module):
    def __init__(
        self,
        character_level_d_model,
        word_level_d_model,
        num_heads_char_encoder,
        num_layers_char_encoder,
        num_heads_word_encoder,
        num_layers_word_encoder,
        dff,
        character_vocab_size,
        word_vocab_size,
        device,
        dropout_rate=0.1,
    ):
        super().__init__()
        self.encoder_character = EncoderCharacter(
            num_heads=num_heads_char_encoder,
            num_layers=num_layers_char_encoder,
            vocab_size=character_vocab_size,
            device=device,
            dff=dff,
            dropout_rate=dropout_rate,
            embed_dim=character_level_d_model,
        ).to(device)
        self.embedding_word = PositionalEmbedding(
            vocab_size=word_vocab_size, d_model=word_level_d_model
        )
        self.encoder_word = nn.ModuleList(
            [
                EncoderLayer(
                    embed_dim=word_level_d_model,
                    num_heads=num_heads_word_encoder,
                    dropout=dropout_rate,
                    dff=dff,
                ).to(device)
                for _ in range(num_layers_word_encoder)
            ]
        ).to(device=device)
        self.linear = nn.Sequential(
            nn.Linear(
                character_level_d_model + word_level_d_model,
                character_level_d_model + word_level_d_model,
            ),
            nn.LayerNorm(
                character_level_d_model + word_level_d_model,
                eps=1e-05,
                elementwise_affine=True,
            ),
            nn.Linear(character_level_d_model + word_level_d_model, word_vocab_size),
            nn.Dropout(p=dropout_rate),
        )

    def forward(self, *inputs):
        # Input
        char_input_ids, word_input_ids, masked_positions = inputs

        # Encode character
        key_mask_padding = word_input_ids != 1
        output_char_encoder = self.encoder_character(char_input_ids, key_mask_padding)
        output_char_encoder = output_char_encoder.view(
            word_input_ids.size(0), word_input_ids.size(1), -1
        )
        # Encode word
        output_word_encode = self.embedding_word(word_input_ids)
        # return 1
        for layer in self.encoder_word:
            output_word_encode = layer(output_word_encode, word_input_ids == 1)
        # Combine encode
        output = torch.cat((output_word_encode, output_char_encoder), -1)
        attn_mask = word_input_ids != 1
        attn_mask = attn_mask[:, None, None, :]
        output = output[masked_positions]
        output = self.linear(output)
        return output


In [17]:
import torch
import torch.nn as nn


class DetectModel(nn.Module):
    def __init__(
        self,
        embed_dim,
        num_heads,
        dff,
        vocab_size,
        num_layer,
        device,
        dropout_rate=0.1,
    ):
        super().__init__()
        self.embedding_layer = PositionalEmbedding(
            vocab_size=vocab_size, d_model=embed_dim
        )
        self.encoder_layers = nn.ModuleList(
            [
                EncoderLayer(
                    embed_dim=embed_dim,
                    num_heads=num_heads,
                    dropout=dropout_rate,
                    dff=dff,
                ).to(device)
                for _ in range(num_layer)
            ]
        )
        self.classifier_layer = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(
                embed_dim,
                eps=1e-05,
                elementwise_affine=True,
            ),
            nn.Linear(embed_dim, 2, bias=True),
            nn.Dropout(p=dropout_rate),
            # nn.Softmax(dim=1),
        )

    def forward(self, x):
        key_mask_padding = x == 1
        x = self.embedding_layer(x)
        for layer in self.encoder_layers:
            x = layer(x, key_mask_padding)
        return self.classifier_layer(x)


### trainning

In [18]:
from torch.utils.data import DataLoader
batch_size = 200
word_embed_dim = 256
char_embed_dim = 256
device0 = torch.device("cuda:0")
device1 = torch.device("cuda:1")
sentences = load_data_and_vocab(
    path_file_data="/kaggle/input/datatrain-sc/sentences.txt",
    data_size=3000,
)
custom_tokenizer = Tokenizer(vocab_file="/kaggle/input/datatrain-sc/vocab.txt", unk_token="<unk>")
char_tokenizer = Tokenizer(
    vocab_file="/kaggle/input/datatrain-sc/char_vocab.txt", unk_token="<unk>", char_level=True
)
data_builder = DataBuilder(
    rng=random.Random(86),
    padding_id=1,
    mask_token="<mask>",
    seq_max_length=32,
    word_max_length=10,
    custom_tokenizer=custom_tokenizer,
    char_tokenizer=char_tokenizer,
)
## Build data
char_input_ids, word_input_ids, mask_token_positions, target = data_builder.build_data(
    inputs=sentences
)
data_train = CustomDataset(
    char_input_ids=char_input_ids,
    word_input_ids=word_input_ids,
    mask_token_positions=mask_token_positions,
    target=target,
)
train_dataloader = DataLoader(
    dataset=data_train,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_train.collate_fn,
)

model_detect = DetectModel(
    embed_dim=768,
    num_heads=4,
    num_layer=12,
    vocab_size=len(custom_tokenizer.vocab),
    device=device1,
    dff=2048,
)

model_fill = ModelSC(
    character_level_d_model=char_embed_dim,
    word_level_d_model=word_embed_dim,
    num_heads_char_encoder=2,
    num_heads_word_encoder=2,
    num_layers_char_encoder=6,
    num_layers_word_encoder=12,
    dff=512,
    character_vocab_size=len(char_tokenizer.vocab),
    word_vocab_size=len(custom_tokenizer.vocab),
    device=device0,
)


def train_loop(
    dataloader,
    model_detect: DetectModel,
    model_fill: ModelSC,
    loss_fn_model_detect,
    loss_fn_model_fill,
    optimizer_model_detect,
    optimizer_model_fill,
    device0,
    device1,
):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model_detect.train()
    model_fill.train()
    for batch, (X, y) in enumerate(dataloader):
        # Data for fill
        word_input_ids_fill: torch.Tensor = X["word_input_ids"].to(device0)
        char_input_ids_fill = X["char_input_ids"].to(device0)
        mask_token_positions_fill = X["mask_token_positions"].to(device0)

        # Train model fill
        pred_fill: torch.Tensor = model_fill(
            char_input_ids_fill, word_input_ids_fill, mask_token_positions_fill
        )
        target_fill = (
            torch.Tensor([item for sublist in y for item in sublist]).long().to(device0)
        )
        loss_fill = loss_fn_model_fill(pred_fill, target_fill)
        loss_fill.backward(retain_graph=True)
        optimizer_model_fill.step()
        optimizer_model_fill.zero_grad()

        # Data for detect
        input_ids_detect = word_input_ids_fill.to(device1)
#         word_input_ids_fill = word_input_ids_fill.to(device1)
        mask_token_positions_fill = mask_token_positions_fill.to(device1)
        pred_fill = pred_fill.to(device1)
        input_ids_detect[mask_token_positions_fill] = pred_fill.argmax(-1)
#         word_input_ids_fill[mask_token_positions_fill] = target_fill
        target_detect = mask_token_positions_fill.to(device1).long()
        target_detect = target_detect.view(
            target_detect.shape[0] * target_detect.shape[1]
        )
        # Train model detect
        pred_detect: torch.Tensor = model_detect(input_ids_detect)
        pred_detect = pred_detect.view(
            pred_detect.shape[0] * pred_detect.shape[1], pred_detect.shape[-1]
        )
        loss_detect = loss_fn_model_detect(pred_detect, target_detect)
        loss_detect.backward()
        optimizer_model_detect.step()
        optimizer_model_detect.zero_grad()
        if batch % 5 == 0:
            loss_fill, current_fill = loss_fill.item(), batch * batch_size
            loss_detect, current_detect = loss_detect.item(), batch * batch_size
            print(
                f"loss: {loss_fill:>7f}  [{current_fill:>5d}/{size:>5d}]",
                f"loss: {loss_detect:>7f}  [{current_detect:>5d}/{size:>5d}]",
            )

def train_loop_fill_masked(
    dataloader,
    model_fill: ModelSC,
    loss_fn_model_fill,
    optimizer_model_fill,
    device,
):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model_fill.train()
    for batch, (X, y) in enumerate(dataloader):
        word_input_ids_fill: torch.Tensor = X["word_input_ids"].to(device)
        char_input_ids_fill = X["char_input_ids"].to(device)
        mask_token_positions_fill = X["mask_token_positions"].to(device)

        # Train model fill
        pred_fill: torch.Tensor = model_fill(
            char_input_ids_fill, word_input_ids_fill, mask_token_positions_fill
        )
        target_fill = (
            torch.Tensor([item for sublist in y for item in sublist]).long().to(device)
        )
        loss_fill = loss_fn_model_fill(pred_fill, target_fill)
        loss_fill.backward(retain_graph=True)
        optimizer_model_fill.step()
        optimizer_model_fill.zero_grad()
        if batch % 5 == 0:
            loss_fill, current_fill = loss_fill.item(), batch * batch_size
            # loss_detect, current_detect = loss_detect.item(), batch * batch_size
            print(
                f"loss: {loss_fill:>7f}  [{current_fill:>5d}/{size:>5d}]",
                # f"loss: {loss_detect:>7f}  [{current_detect:>5d}/{size:>5d}]",
            )
            # for name, param in model_fill.named_parameters():
            #     if param.grad is not None:
            #         print(f"Grad for {name}: {param.grad.norm().item()}")

            # # Check if weights are updating
            # for name, param in model_fill.named_parameters():
            #     print(f"Weight for {name}: {param.norm().item()}")


In [19]:
# from prettytable import PrettyTable

# def count_parameters(model):
#     table = PrettyTable(["Modules", "Parameters"])
#     total_params = 0
#     for name, parameter in model.named_parameters():
#         if not parameter.requires_grad:
#             continue
#         params = parameter.numel()
#         table.add_row([name, params])
#         total_params += params
#     print(table)
#     print(f"Total Trainable Params: {total_params}")
#     return total_params

# count_parameters(model_fill)

In [20]:
learning_rate = 1e-3
epochs = 20
optimizer_fill = torch.optim.Adam(model_fill.parameters(), lr=learning_rate)
optimizer_detect = torch.optim.Adam(model_detect.parameters(), lr=learning_rate)
loss_fn_fill = torch.nn.CrossEntropyLoss()
loss_fn_detect = torch.nn.CrossEntropyLoss()
model_fill.to(device0)
model_detect.to(device1)
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(
        model_detect=model_detect,
        model_fill=model_fill,
        dataloader=train_dataloader,
        loss_fn_model_detect=loss_fn_detect,
        loss_fn_model_fill=loss_fn_fill,
        optimizer_model_detect=optimizer_detect,
        optimizer_model_fill=optimizer_fill,
        device0=device0,
        device1=device1,
    )
print("Done!")

Epoch 1
-------------------------------
loss: 9.309552  [    0/40538] loss: 1.065825  [    0/40538]
loss: 6.895817  [ 1000/40538] loss: 1.167748  [ 1000/40538]
loss: 6.505784  [ 2000/40538] loss: 0.379361  [ 2000/40538]
loss: 6.255627  [ 3000/40538] loss: 0.416609  [ 3000/40538]
loss: 6.175040  [ 4000/40538] loss: 0.379793  [ 4000/40538]
loss: 6.285351  [ 5000/40538] loss: 0.381466  [ 5000/40538]
loss: 6.087528  [ 6000/40538] loss: 0.364450  [ 6000/40538]
loss: 6.127000  [ 7000/40538] loss: 0.371301  [ 7000/40538]
loss: 5.875450  [ 8000/40538] loss: 0.359280  [ 8000/40538]
loss: 6.022274  [ 9000/40538] loss: 0.381149  [ 9000/40538]
loss: 5.777339  [10000/40538] loss: 0.370257  [10000/40538]
loss: 5.833948  [11000/40538] loss: 0.369862  [11000/40538]
loss: 5.733826  [12000/40538] loss: 0.362514  [12000/40538]
loss: 5.569263  [13000/40538] loss: 0.361447  [13000/40538]
loss: 5.561000  [14000/40538] loss: 0.365304  [14000/40538]
loss: 5.774527  [15000/40538] loss: 0.371647  [15000/40538]


In [23]:
text = "Giá như hôm đó không tỏ tình thì có lẽ bầu trời vẫn còn xanh"
(char_input, word_input, mask_pos, target) = data_builder.build_data([text])

In [25]:
rere = model_fill(char_input.to(device0), word_input.to(device0), mask_pos.to(device0))

In [28]:
target

[[5160, 3247],
 [1280, 8065],
 [7220, 704],
 [5370, 2485],
 [7301, 3028],
 [4229, 2102],
 [7095, 530],
 [7095, 530]]

In [27]:
rere.argmax(-1)

tensor([5160, 4808, 1280, 8065, 7220,  704, 5370, 2485, 7301, 2771, 4229, 2102,
        7095,  338, 7095,  530], device='cuda:0')